In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Download the dataset
!wget -O poetry.txt https://drive.google.com/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

--2023-12-24 21:54:38--  https://drive.google.com/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT
Resolving drive.google.com (drive.google.com)... 74.125.139.113, 74.125.139.138, 74.125.139.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.139.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/je81paiugeaa4d2keeo4vft2m6bcrn5l/1703454825000/08094225471505108399/*/15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT?uuid=b376ee84-3254-4a17-b62f-786e0a73e693 [following]
--2023-12-24 21:54:38--  https://doc-10-58-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/je81paiugeaa4d2keeo4vft2m6bcrn5l/1703454825000/08094225471505108399/*/15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT?uuid=b376ee84-3254-4a17-b62f-786e0a73e693
Resolving doc-10-58-docs.googleusercontent.com (doc-10-58-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-1

In [3]:
!rm /kaggle/working/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

rm: cannot remove '/kaggle/working/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT': No such file or directory


In [4]:
with open("poetry.txt", 'r') as f:
    data = f.read()
    
corpus = data.lower().split("\n")

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1

In [7]:
embedding_dim = 16
lstm_dim = 64
dense_dim = 64
epochs=500

In [8]:
input_sequences = []

for line in corpus:
    sequence = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(sequence)):
        n_gram_sequence = sequence[:i+1]
        input_sequences.append(n_gram_sequence)

In [9]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, truncating="pre", padding="pre")

In [10]:
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [11]:
model = keras.Sequential([
    keras.layers.Embedding(total_words, embedding_dim, input_length=max_sequence_length-1),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_dim, return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(lstm_dim)),
    keras.layers.Dense(dense_dim, activation='relu'),
    keras.layers.Dense(total_words, activation='sigmoid')
])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [13]:
history = model.fit(xs, ys, epochs=epochs)

Epoch 1/500
377/377 [==============================] - 9s 24ms/step - loss: 6.4794 - accuracy: 0.0657
Epoch 2/500
377/377 [==============================] - 9s 24ms/step - loss: 6.2531 - accuracy: 0.0670
Epoch 3/500
377/377 [==============================] - 9s 24ms/step - loss: 6.0596 - accuracy: 0.0782
Epoch 4/500
377/377 [==============================] - 9s 24ms/step - loss: 5.9286 - accuracy: 0.0804
Epoch 5/500
377/377 [==============================] - 9s 24ms/step - loss: 5.8121 - accuracy: 0.0824
Epoch 6/500
377/377 [==============================] - 9s 24ms/step - loss: 5.7128 - accuracy: 0.0858
Epoch 7/500
377/377 [==============================] - 9s 24ms/step - loss: 5.6211 - accuracy: 0.0901
Epoch 8/500
377/377 [==============================] - 9s 25ms/step - loss: 5.5371 - accuracy: 0.0956
Epoch 9/500
377/377 [==============================] - 9s 24ms/step - loss: 5.4635 - accuracy: 0.0989
Epoch 10/500
377/377 [==============================] - 9s 24ms/step - loss: 5.392

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



377/377 [==============================] - 9s 23ms/step - loss: 0.5396 - accuracy: 0.8440
Epoch 445/500
377/377 [==============================] - 9s 23ms/step - loss: 0.5018 - accuracy: 0.8540
Epoch 446/500
377/377 [==============================] - 9s 23ms/step - loss: 0.4956 - accuracy: 0.8542
Epoch 447/500
377/377 [==============================] - 9s 24ms/step - loss: 0.4935 - accuracy: 0.8546
Epoch 448/500
377/377 [==============================] - 9s 23ms/step - loss: 0.5064 - accuracy: 0.8525
Epoch 449/500
377/377 [==============================] - 9s 23ms/step - loss: 0.5669 - accuracy: 0.8401
Epoch 450/500
377/377 [==============================] - 9s 24ms/step - loss: 0.5890 - accuracy: 0.8334
Epoch 451/500
377/377 [==============================] - 9s 23ms/step - loss: 0.6037 - accuracy: 0.8277
Epoch 452/500
377/377 [==============================] - 9s 24ms/step - loss: 0.5603 - accuracy: 0.8393
Epoch 453/500
377/377 [==============================] - 9s 23ms/step - loss: 

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
seed_text = "help me obi-wan kinobi youre my only hope"
gen_word_count = 10

for _ in range(gen_word_count):
    #seed_seq = tokenizer.texts_to_sequences([seed_text])
    #padded_seed_seq = pad_sequences([seed_seq], padding='pre', maxlen=max_sequence_length)
    
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
    
    probabilities = model.predict([token_list], verbose=0)
    predicted = np.argmax(probabilities, axis=-1)[0]
    if predicted != 0:
        # Look up the word associated with the index. 
        output_word = tokenizer.index_word[predicted]
        
        # Combine with the seed text
        seed_text += " " + output_word

# Print the result	
print(seed_text)

help me obi-wan kinobi youre my only hope him did please was to the spire it fountain would
